In [1]:

# Linear programming (LP), also called linear optimization, is a method to achieve the best outcome (such as 
# maximum profit or lowest cost) in a mathematical model whose requirements are represented by linear relationships.

# Formally, linear programming is a technique for the optimization of a linear objective function, 
# subject to linear equality and linear inequality constraints. 


In [1]:


import pulp

# Define the optimization problem
problem = pulp.LpProblem("Worker Scheduling", pulp.LpMinimize)

# Define the decision variables
shifts = ['Morning', 'Afternoon', 'Evening']
workers = ['W1', 'W2', 'W3', 'W4', 'W5']

# Define the cost of each worker for each shift
cost = {
    ('W1', 'Morning'): 500,
    ('W1', 'Afternoon'): 550,
    ('W1', 'Evening'): 200,
    ('W2', 'Morning'): 120,
    ('W2', 'Afternoon'): 170,
    ('W2', 'Evening'): 210,
    ('W3', 'Morning'): 110,
    ('W3', 'Afternoon'): 160,
    ('W3', 'Evening'): 200,
    ('W4', 'Morning'): 130,
    ('W4', 'Afternoon'): 180,
    ('W4', 'Evening'): 220,
    ('W5', 'Morning'): 90,
    ('W5', 'Afternoon'): 140,
    ('W5', 'Evening'): 180
}

# Define the demand for each shift
demand = {
    'Morning': 3,
    'Afternoon': 4,
    'Evening': 5
}

# Define the decision variables
shift_vars = pulp.LpVariable.dicts("Shift", (workers, shifts), cat='Binary')

# Define the objective function
problem += pulp.lpSum([cost[(w, s)] * shift_vars[w][s] for w in workers for s in shifts])

# Define the constraints
for s in shifts:
    problem += pulp.lpSum([shift_vars[w][s] for w in workers]) >= demand[s]

# Solve the problem
problem.solve()

# Print the results
for w in workers:
    for s in shifts:
        if shift_vars[w][s].value() == 1:
            print(f"{w} works the {s} shift")


C:\Users\ryans\anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


W1 works the Evening shift
W2 works the Morning shift
W2 works the Afternoon shift
W2 works the Evening shift
W3 works the Morning shift
W3 works the Afternoon shift
W3 works the Evening shift
W4 works the Afternoon shift
W4 works the Evening shift
W5 works the Morning shift
W5 works the Afternoon shift
W5 works the Evening shift


In [3]:

print(f"\nTotal minimum cost: ${problem.objective.value():,.2f}")



Total minimum cost: $1,980.00


In [4]:

# Let's re-try the concept using a slightly different approach.

from pyomo.environ import *
import pyomo.environ as pyo

# define parameters
num_workers = 10
num_shifts = 3
market_expenses = [1000, 2000, 3000]

# define the model
model = pyo.ConcreteModel()

# define decision variables
model.x = Var(range(num_workers), range(num_shifts), within=Binary)

# define objective function
model.obj = Objective(expr=sum(market_expenses[s] * sum(model.x[w, s] for w in range(num_workers)) for s in range(num_shifts)), sense=minimize)

# define constraints
model.worker_constraint = ConstraintList()
for w in range(num_workers):
    model.worker_constraint.add(sum(model.x[w, s] for s in range(num_shifts)) == 1)

model.shift_constraint = ConstraintList()
for s in range(num_shifts):
    model.shift_constraint.add(sum(model.x[w, s] for w in range(num_workers)) >= 2)

# solve the optimization problem
solver = SolverFactory('glpk')
solver.solve(model)


# print the results
for w in range(num_workers):
    for s in range(num_shifts):
        if value(model.x[w, s]) > 0.5:
            print("Worker %d is assigned to shift %d" % (w, s))


Worker 0 is assigned to shift 0
Worker 1 is assigned to shift 2
Worker 2 is assigned to shift 2
Worker 3 is assigned to shift 0
Worker 4 is assigned to shift 1
Worker 5 is assigned to shift 1
Worker 6 is assigned to shift 0
Worker 7 is assigned to shift 0
Worker 8 is assigned to shift 0
Worker 9 is assigned to shift 0


In [7]:

import pandas as pd
import pulp
from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, lpSum, LpVariable
import numpy as np


# load CSV into dataframe
df = pd.read_csv("C:\\workers_schedules.csv")
df.head()


,Time Windows,Shift 1,Shift 2,Shift 3,Shift 4,Workers Required
0,6:00 - 9:00,X,NaN,NaN,X,55.0
1,9:00 - 12:00,X,NaN,NaN,NaN,46.0
2,12:00 - 15:00,X,X,NaN,NaN,59.0
3,15:00 - 18:00,NaN,X,NaN,NaN,23.0
4,18:00 - 21:00,NaN,X,X,NaN,60.0


In [8]:

df = df.fillna(0).applymap(lambda x: 1 if x == "X" else x)

#df.set_index('Time Windows')
a = df.drop(columns=['Time Windows', 'Workers Required']).values
a 


array([[1, 0, 0, 1],
       [1, 0, 0, 0],
       [1, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 1],
       [0, 0, 0, 1],
       ['135', '140', '190', '188']], dtype=object)

In [9]:

# number of shifts 
n = a.shape[1]
n


4

In [10]:

# number of time windows minus last row
# this can be derived different ways
T = a.shape[0]-1
T


8

In [11]:

# number of workers required per time window
d = df["Workers Required"].values
d


array([55., 46., 59., 23., 60., 38., 20., 30.,  0.])

In [12]:

# wage rate per shift
#Get last row of dataframe
last_row = df.iloc[-1:,1:-1]
#Get last row of dataframe as numpy array
w = last_row.to_numpy()
w


array([['135', '140', '190', '188']], dtype=object)

In [13]:

# Decision variables
y = LpVariable.dicts("num_workers", list(range(n)), lowBound=0, cat="Integer")
y


{0: num_workers_0, 1: num_workers_1, 2: num_workers_2, 3: num_workers_3}

In [14]:

# Create problem
prob = LpProblem("scheduling_workers", LpMinimize)


for t in range(T):
    prob += lpSum([a[t, j] * y[j] for j in range(n)]) >= d[t]


prob.solve()
print("Status:", LpStatus[prob.status])


Status: Optimal


In [15]:

for shift in range(n):
    print(f"The number of workers needed for shift {shift} is {int(y[shift].value())} workers")


The number of workers needed for shift 0 is 46 workers
The number of workers needed for shift 1 is 23 workers
The number of workers needed for shift 2 is 38 workers
The number of workers needed for shift 3 is 30 workers
